Недооцененная по сложности задача, решение которой хорошо описанно [тут](https://habr.com/ru/companies/yandex/articles/414997/). В итоге, я ориентировался на это же решение и подробнее не опишу. Возможно, кому-то будет понятнее мой код без использования классов (но вряд ли). Также ниже ещё один подход к задаче.

In [ ]:
import sys

n = int(sys.stdin.readline())
xy = []
right_n = 0
sum_right = 0
right_sse = 0
for _ in range(n):
    x,y = map(float,sys.stdin.readline().split())
    xy.append([x,y])
    sum_right += y
    right_n += 1
    b = sum_right/right_n
    right_diff = y - b
    right_sse += right_diff*(y-b)


xy.sort()

a = 0
sum_left = 0
left_n = 0
left_sse = 0

min_mse = right_sse

min_c = xy[0][0]
min_b = sum_right/right_n
min_a = 0

for i in range(len(xy)-1):
    cur_item = xy[i]
    next_item = xy[i+1]
    c = (cur_item[0]+next_item[0])/2
    sum_left += cur_item[1]
    sum_right -= cur_item[1]
    left_n += 1
    right_n -= 1
    left_diff = cur_item[1] - a
    right_diff = cur_item[1] - b
    a = sum_left/left_n
    b = sum_right/right_n
    left_sse += left_diff*(cur_item[1]-a)
    right_sse -= right_diff*(cur_item[1]-b)
    if cur_item[0] == next_item[0]:
        continue

    temp_mse = left_sse + right_sse
    if temp_mse<min_mse:
        min_mse = temp_mse
        min_a,min_b,min_c = a,b,c

print(min_a,min_b,min_c)

Решение от [Кобелева Льва](https://github.com/evlko).

Идея:
1. Берем только уникальные элементы (пары), нет смысла хранить две одинаковые пары `(x, y)`;
2. Далее сортируем массив `x`, так как очев, что будем пробегаться именно по нему
3. Если вдруг в `X` или `Y` только 1 уникальный эл, то смысла запускать цикл подбора `threshold` нет
4. Иначе начинаем искать разбиние с 1-ого элемента
* Берем среднее между i и i-1 элами, так как у нас целые числа;
* Если i-1 эл больше (лол) или равен, то идем далее
* Иначе берем сумму слева от треша, если быть точнее, то мы ранее посчитали `cumsum`, поэтому просто берем из массива `i-1` эл
* Далее мы стараемся **максимизировать** правую часть RMSE без корня, так как минимизация корня приводит к минимизации не корня, а так как у нас все значения больше нуля и мы получаем A - (B + C), где A - иском, а B и C - ошибки, то это тоже самое, что стараться максимизировать B + C, чтобы A - (B + C) стремилось к нулю (без ошибок)
* если текущая ошибка **больше**, то записываем данные

Далее просто проверяем разные эдж кейсы

In [ ]:
import sys
import numpy as np

def main():
    n = int(input())
    X = np.zeros(n)
    Y = np.zeros(n)

    unique_values = set()
    sum_total = 0
    arr_index = 0
    for i in range(n):
        x, y = map(int, sys.stdin.readline().split())
        if (x, y) not in unique_values:
            sum_total += y
            X[arr_index], Y[arr_index] = x, y
            unique_values.add((x, y))
            arr_index += 1

    X = X[:arr_index]
    Y = Y[:arr_index]

    sort_indices = np.argsort(X)

    X = X[sort_indices]
    Y = Y[sort_indices]

    best_threshold, best_rmse = None, float('-inf')
    prev_threshold = None

    flag = True
    if len(set(X)) == 1 or len(set(Y)) == 1:
        flag = False
    else:
        sum_left_cs = np.cumsum(Y)

    if flag:
        n = len(X)
        for i in range(1, n):
            threshold = (X[i-1] + X[i]) / 2

            if X[i-1] >= threshold:
                continue

            sum_left = sum_left_cs[i-1]
            rmse = (sum_left**2) / i + ((sum_total - sum_left)**2) / (n-i)

            if rmse > best_rmse:
                best_rmse, best_threshold = rmse, threshold

    if best_rmse != float('-inf'):
        left_class, right_class = np.mean(Y[X < best_threshold]), np.mean(Y[X >= best_threshold])
    else:
        best_threshold = X[0] - 0.5
        left_class, right_class = -np.mean(Y[X >= best_threshold]), np.mean(Y[X >= best_threshold])
    if left_class == right_class:
        best_threshold = min(X) - 0.5
        if right_class == 0:
            left_class = -1
        else:
            left_class *= -1

    if np.isnan(left_class):
        if right_class == 0:
            left_class = -1
        else:
            left_class = right_class * -1
        best_threshold -= 0.5

    print(left_class, right_class, best_threshold)

if __name__ == '__main__':
    main()